<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_class_06/blob/main/T5_TREC_COVID_expansion_qualitative_test_for_doc2query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook allows qualitative analysis of the generated topics considering some of the trained models

In [1]:
!pip install transformers -q
!pip install evaluate -q
!pip install ftfy -q
!pip install sentencepiece -q
!pip install sacrebleu -q
!pip install comet_ml -q

In [2]:
WORKING_FOLDER="drive/MyDrive/unicamp/ia368v_dd/aula_06"

API_KEYS_FILE="/content/drive/MyDrive/unicamp/ia368v_dd/api_keys_20230324.json"

TRAIN_OUTPUT_FOLDER="./trec_covid_expansion"

TREC_COVID_MERGED_DATA_FILENAME="trec_covid_merged_data.tsv"
TREC_COVID_CORPUS_FILENAME="trec_covid_corpus.tsv"

LINK_WITH_COMET=False

In [3]:
import os
from google.colab import drive
import json

import ftfy
import pandas as pd
import numpy as np

from scipy import stats

import pickle

import torch

In [4]:
drive.mount('/content/drive', force_remount=True)
os.chdir(WORKING_FOLDER)

Mounted at /content/drive


In [5]:
from transformers import (AutoTokenizer, 
                          AutoModelForSeq2SeqLM, 
                          Seq2SeqTrainer,
                          Seq2SeqTrainingArguments,

                          GenerationConfig,

                          TrainerCallback, 
                          get_cosine_with_hard_restarts_schedule_with_warmup,
                          DataCollatorForSeq2Seq,

                          )

import torch

import evaluate

comet_ml is installed but `COMET_API_KEY` is not set.


In [6]:
pd.set_option('display.max_colwidth', None)

## Read the TREC COVID merged data

This data should have been prepared by the `explore_trec_covid.ipynb` notebook.

In [7]:
trec_covid_corpus_df = pd.read_csv(TREC_COVID_CORPUS_FILENAME, sep='\t')

# Prepare T5 model

Load the T5 mode fine-tuned to the doc2query document expansion task.

In [44]:
BEST_FINE_TUNED_T5_MODEL="trained_model/checkpoint-100-19.6221"
BEST_FINE_TUNED_T5_MODEL_MORE_DATA="trained_model_more_data/checkpoint-150-17.8306"
LONGER_FINE_TUNED_T5_MODEL="trained_model/checkpoint-3000"

In [45]:
MODELS_TO_TEST=[BEST_FINE_TUNED_T5_MODEL, BEST_FINE_TUNED_T5_MODEL_MORE_DATA, LONGER_FINE_TUNED_T5_MODEL]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
instantiated_models = []

for which_model in MODELS_TO_TEST:
    instantiated_models.append(AutoModelForSeq2SeqLM.from_pretrained(which_model).to(device))

In [56]:
def generate_test_sequences(original_sequences, models_to_test, generation_params, model_names):
    generated_sequences = []

    for i, test_sequence in enumerate(original_sequences):
        for j, which_model in enumerate(models_to_test):
            print("Generating topics for sequence={}, model={}".format(i, model_names[j]))

            input_ids = tokenizer(test_sequence, return_tensors='pt').input_ids.to(device)

            generated_text = which_model.generate(inputs=input_ids, generation_config=generation_params)

            decoded_text = tokenizer.batch_decode(generated_text, skip_special_tokens=True)

            generated_sequences += list(zip([test_sequence] * len(decoded_text), 
                                            [model_names[j]] * len(decoded_text), 
                                            decoded_text))
            

    return generated_sequences

In [25]:
generation_params = GenerationConfig(max_new_tokens=100, 
                                     do_sample=True, 
                                     temperature=1.5,
                                     top_p=0.9,
                                     num_beams=10, 
                                     num_return_sequences=10)

In [57]:
r1 = generate_test_sequences(trec_covid_corpus_df.iloc[[68547, 2000]]['text'].tolist(), instantiated_models, generation_params, MODELS_TO_TEST)

Generating topics for sequence=0, model=trained_model/checkpoint-100-19.6221
Generating topics for sequence=0, model=trained_model_more_data/checkpoint-150-17.8306
Generating topics for sequence=0, model=trained_model/checkpoint-3000
Generating topics for sequence=1, model=trained_model/checkpoint-100-19.6221
Generating topics for sequence=1, model=trained_model_more_data/checkpoint-150-17.8306
Generating topics for sequence=1, model=trained_model/checkpoint-3000


In [58]:
pd.DataFrame(r1, columns=['sequence', 'model', 'topics'])

,sequence,model,topics
0,"Coronavirus disease 2019 (COVID-19) can be screened and diagnosed through the detection of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) by real-time reverse transcription polymerase chain reaction. SARS-CoV-2 nucleic acid amplification tests (NAATs) have been rapidly developed and quickly applied to clinical testing during the pandemic. However, studies evaluating the performance of these NAAT assays are limited. We evaluated the performance of four NAATs, which were marked by the Conformité Européenne and widely used in China during the pandemic. Results showed that the analytical sensitivity of the four assays was significantly lower than that claimed by the NAAT manufacturers. The limit of detection (LOD) of Daan, Sansure, and Hybribio NAATs was 3000 copies/mL, whereas the LOD of Bioperfectus NAATs was 4000 copies/mL. The results of the consistency test using 46 samples showed that Daan, Sansure, and Hybribio NAATs could detect the samples with a specificity of 100% (30/30) and a sensitivity of 100% (16 /16), whereas Bioperfectus NAAT detected the samples with a specificity of 100% (30/30) and a sensitivity 81.25% (13/16). The sensitivity of Bioperfectus NAAT was lower than that of the three other NAATs; this finding was consistent with the result that Bioperfectus NAAT had a higher LOD than the three other kinds of NAATs. The four above mentioned reagents presented high specificity; however, for the detection of the samples with low virus concentration, Bioperfectus reagent had the risk of missing detection. Therefore, the LOD should be considered in the selection of SARS-CoV-2 NAATs.",trained_model/checkpoint-100-19.6221,what is the lod of a typical sars cov 2 naat
1,"Coronavirus disease 2019 (COVID-19) can be screened and diagnosed through the detection of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) by real-time reverse transcription polymerase chain reaction. SARS-CoV-2 nucleic acid amplification tests (NAATs) have been rapidly developed and quickly applied to clinical testing during the pandemic. However, studies evaluating the performance of these NAAT assays are limited. We evaluated the performance of four NAATs, which were marked by the Conformité Européenne and widely used in China during the pandemic. Results showed that the analytical sensitivity of the four assays was significantly lower than that claimed by the NAAT manufacturers. The limit of detection (LOD) of Daan, Sansure, and Hybribio NAATs was 3000 copies/mL, whereas the LOD of Bioperfectus NAATs was 4000 copies/mL. The results of the consistency test using 46 samples showed that Daan, Sansure, and Hybribio NAATs could detect the samples with a specificity of 100% (30/30) and a sensitivity of 100% (16 /16), whereas Bioperfectus NAAT detected the samples with a specificity of 100% (30/30) and a sensitivity 81.25% (13/16). The sensitivity of Bioperfectus NAAT was lower than that of the three other NAATs; this finding was consistent with the result that Bioperfectus NAAT had a higher LOD than the three other kinds of NAATs. The four above mentioned reagents presented high specificity; however, for the detection of the samples with low virus concentration, Bioperfectus reagent had the risk of missing detection. Therefore, the LOD should be considered in the selection of SARS-CoV-2 NAATs.",trained_model/checkpoint-100-19.6221,what is the difference between sars nats and bioperfectus nats
2,"Coronavirus disease 2019 (COVID-19) can be screened and diagnosed through the detection of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) by real-time reverse transcription polymerase chain reaction. SARS-CoV-2 nucleic acid amplification tests (NAATs) have been rapidly developed and quickly applied to clinical testing during the pandemic. However, studies evaluating the performance of these NAAT assays are limited. We evaluated the performance of four NAATs, which were marked by the Conformité Europée

In [59]:
r2 = generate_test_sequences(trec_covid_corpus_df.iloc[[2, 3]]['text'].tolist(), instantiated_models, generation_params, MODELS_TO_TEST)

Generating topics for sequence=0, model=trained_model/checkpoint-100-19.6221
Generating topics for sequence=0, model=trained_model_more_data/checkpoint-150-17.8306
Generating topics for sequence=0, model=trained_model/checkpoint-3000
Generating topics for sequence=1, model=trained_model/checkpoint-100-19.6221
Generating topics for sequence=1, model=trained_model_more_data/checkpoint-150-17.8306
Generating topics for sequence=1, model=trained_model/checkpoint-3000


In [60]:
pd.DataFrame(r2, columns=['sequence', 'model', 'topics'])

,sequence,model,topics
0,"Surfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung. SP-D is synthesized and secreted by alveolar and bronchiolar epithelial cells, but is also expressed by epithelial cells lining various exocrine ducts and the mucosa of the gastrointestinal and genitourinary tracts. SP-D, a collagenous calcium-dependent lectin (or collectin), binds to surface glycoconjugates expressed by a wide variety of microorganisms, and to oligosaccharides associated with the surface of various complex organic antigens. SP-D also specifically interacts with glycoconjugates and other molecules expressed on the surface of macrophages, neutrophils, and lymphocytes. In addition, SP-D binds to specific surfactant-associated lipids and can influence the organization of lipid mixtures containing phosphatidylinositol in vitro. Consistent with these diverse in vitro activities is the observation that SP-D-deficient transgenic mice show abnormal accumulations of surfactant lipids, and respond abnormally to challenge with respiratory viruses and bacterial lipopolysaccharides. The phenotype of macrophages isolated from the lungs of SP-D-deficient mice is altered, and there is circumstantial evidence that abnormal oxidant metabolism and/or increased metalloproteinase expression contributes to the development of emphysema. The expression of SP-D is increased in response to many forms of lung injury, and deficient accumulation of appropriately oligomerized SP-D might contribute to the pathogenesis of a variety of human lung diseases.",trained_model/checkpoint-100-19.6221,what is the phenotype of a sp-d deficiency in lungs
1,"Surfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung. SP-D is synthesized and secreted by alveolar and bronchiolar epithelial cells, but is also expressed by epithelial cells lining various exocrine ducts and the mucosa of the gastrointestinal and genitourinary tracts. SP-D, a collagenous calcium-dependent lectin (or collectin), binds to surface glycoconjugates expressed by a wide variety of microorganisms, and to oligosaccharides associated with the surface of various complex organic antigens. SP-D also specifically interacts with glycoconjugates and other molecules expressed on the surface of macrophages, neutrophils, and lymphocytes. In addition, SP-D binds to specific surfactant-associated lipids and can influence the organization of lipid mixtures containing phosphatidylinositol in vitro. Consistent with these diverse in vitro activities is the observation that SP-D-deficient transgenic mice show abnormal accumulations of surfactant lipids, and respond abnormally to challenge with respiratory viruses and bacterial lipopolysaccharides. The phenotype of macrophages isolated from the lungs of SP-D-deficient mice is altered, and there is circumstantial evidence that abnormal oxidant metabolism and/or increased metalloproteinase expression contributes to the development of emphysema. The expression of SP-D is increased in response to many forms of lung injury, and deficient accumulation of appropriately oligomerized SP-D might contribute to the pathogenesis of a variety of human lung diseases.",trained_model/checkpoint-100-19.6221,what is the role of a surfactant protein in pulmonary emphysema
2,"Surfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung. SP-D is synthesized and secreted by alveolar and bronchiolar epithelial cells, but is also expressed by epithelial cells lining various exocrine ducts and the mucosa of the gastrointestinal and genitourinary tracts. SP-D, a collagenous calcium-dependent lectin (or collectin), binds to surface gly